These queries provide comprehensive insights into:

Customer segmentation based on value and behavior
Purchase patterns including frequency, recency, and basket composition
Digital engagement across channels and conversion funnels
Personalization effectiveness through affinity scores
Retention and lifecycle management opportunities
Size/fit optimization to reduce returns

Each query is optimized for your star schema and provides actionable insights for business decisions. Would you like me to create additional queries focused on specific customer behavior aspects or create a dashboard-ready view combining these metrics?

# 1 📊 Customer Segmentation & Profiling

In [0]:
-- 1.1 Customer Lifetime Value Distribution with Behavior Patterns
WITH customer_metrics AS (
    SELECT 
        c.customer_key,
        c.segment,
        c.lifetime_value,
        c.preferred_channel,
        c.acquisition_channel,
        COUNT(DISTINCT s.transaction_id) as total_orders,
        SUM(s.net_sales_amount) as total_spent,
        AVG(s.net_sales_amount) as avg_order_value,
        DATEDIFF(MAX(d.calendar_date), MIN(d.calendar_date)) as customer_lifespan_days,
        COUNT(DISTINCT d.month) as active_months,
        SUM(CASE WHEN s.is_return THEN 1 ELSE 0 END) * 100.0 / COUNT(*) as return_rate
    FROM juan_dev.retail.gold_customer_dim c
    LEFT JOIN juan_dev.retail.gold_sales_fact s ON c.customer_key = s.customer_key
    LEFT JOIN juan_dev.retail.gold_date_dim d ON s.date_key = d.date_key
    WHERE c.is_current = true
    GROUP BY c.customer_key, c.segment, c.lifetime_value, c.preferred_channel, c.acquisition_channel
),
percentiles AS (
    SELECT 
        PERCENTILE_CONT(0.25) WITHIN GROUP (ORDER BY lifetime_value) as ltv_p25,
        PERCENTILE_CONT(0.50) WITHIN GROUP (ORDER BY lifetime_value) as ltv_p50,
        PERCENTILE_CONT(0.75) WITHIN GROUP (ORDER BY lifetime_value) as ltv_p75,
        PERCENTILE_CONT(0.90) WITHIN GROUP (ORDER BY lifetime_value) as ltv_p90
    FROM customer_metrics
)
SELECT 
    CASE 
        WHEN cm.lifetime_value >= p.ltv_p90 THEN 'VIP (Top 10%)'
        WHEN cm.lifetime_value >= p.ltv_p75 THEN 'High Value (75-90%)'
        WHEN cm.lifetime_value >= p.ltv_p50 THEN 'Mid Value (50-75%)'
        WHEN cm.lifetime_value >= p.ltv_p25 THEN 'Low Value (25-50%)'
        ELSE 'Entry Level (Bottom 25%)'
    END as value_tier,
    cm.segment,
    COUNT(*) as customer_count,
    AVG(cm.total_orders) as avg_orders,
    AVG(cm.total_spent) as avg_revenue,
    AVG(cm.avg_order_value) as avg_basket_size,
    AVG(cm.customer_lifespan_days) as avg_lifespan_days,
    AVG(cm.return_rate) as avg_return_rate
FROM customer_metrics cm
CROSS JOIN percentiles p
GROUP BY 1, 2
ORDER BY 1, 2;

In [0]:
-- 1.2 Customer Journey & Channel Migration Analysis
WITH channel_journey AS (
    SELECT 
        c.customer_key,
        c.acquisition_channel,
        c.preferred_channel,
        s.channel_key,
        ch.channel_category,
        d.fiscal_quarter,
        d.fiscal_year,
        ROW_NUMBER() OVER (PARTITION BY c.customer_key ORDER BY d.calendar_date) as purchase_sequence,
        COUNT(*) OVER (PARTITION BY c.customer_key) as total_purchases
    FROM juan_dev.retail.gold_customer_dim c
    JOIN juan_dev.retail.gold_sales_fact s ON c.customer_key = s.customer_key
    JOIN juan_dev.retail.gold_channel_dim ch ON s.channel_key = ch.channel_key
    JOIN juan_dev.retail.gold_date_dim d ON s.date_key = d.date_key
    WHERE c.is_current = true
)
SELECT 
    acquisition_channel,
    preferred_channel,
    channel_category as current_channel,
    COUNT(DISTINCT customer_key) as customers,
    AVG(CASE WHEN purchase_sequence = 1 THEN 1.0 ELSE 0 END) * 100 as pct_first_purchase,
    AVG(CASE WHEN purchase_sequence = total_purchases THEN 1.0 ELSE 0 END) * 100 as pct_last_purchase,
    AVG(total_purchases) as avg_purchases_per_customer
FROM channel_journey
GROUP BY acquisition_channel, preferred_channel, channel_category
HAVING COUNT(*) > 5
ORDER BY customers DESC;

# 2 🛒 Purchase Behavior Analysis

In [0]:
-- 2.1 Purchase Frequency & Recency Segmentation (RFM-style)
WITH customer_purchases AS (
    SELECT 
        c.customer_key,
        c.customer_id,
        c.segment,
        s.transaction_id,
        d.calendar_date,
        s.net_sales_amount,
        LEAD(d.calendar_date) OVER (
            PARTITION BY c.customer_key 
            ORDER BY d.calendar_date
        ) as next_purchase_date
    FROM juan_dev.retail.gold_customer_dim c
    LEFT JOIN juan_dev.retail.gold_sales_fact s ON c.customer_key = s.customer_key
    LEFT JOIN juan_dev.retail.gold_date_dim d ON s.date_key = d.date_key
    WHERE c.is_current = true
        AND s.is_return = false
),
customer_rfm AS (
    SELECT
        customer_key,
        customer_id,
        segment,
        COUNT(DISTINCT transaction_id) as frequency,
        DATEDIFF(CURRENT_DATE, MAX(calendar_date)) as recency_days,
        SUM(net_sales_amount) as monetary_value,
        AVG(DATEDIFF(next_purchase_date, calendar_date)) as avg_days_between_purchases
    FROM customer_purchases
    GROUP BY customer_key, customer_id, segment
),
rfm_scores AS (
    SELECT 
        *,
        NTILE(5) OVER (ORDER BY recency_days DESC) as r_score,  -- Lower recency = higher score
        NTILE(5) OVER (ORDER BY frequency) as f_score,
        NTILE(5) OVER (ORDER BY monetary_value) as m_score
    FROM customer_rfm
),
rfm_labeled AS (
    SELECT 
        *,
        CASE 
            WHEN r_score >= 4 AND f_score >= 4 AND m_score >= 4 THEN 'Champions'
            WHEN r_score >= 3 AND f_score >= 3 AND m_score >= 4 THEN 'Loyal Customers'
            WHEN r_score >= 3 AND f_score <= 2 AND m_score >= 3 THEN 'Potential Loyalists'
            WHEN r_score >= 4 AND f_score <= 2 THEN 'New Customers'
            WHEN r_score <= 2 AND f_score >= 3 THEN 'At Risk'
            WHEN r_score <= 2 AND f_score <= 2 AND m_score >= 3 THEN 'Cant Lose Them'
            WHEN r_score <= 2 AND f_score <= 2 AND m_score <= 2 THEN 'Lost'
            ELSE 'Others'
        END as rfm_segment
    FROM rfm_scores
)
SELECT 
    rfm_segment,
    COUNT(*) as customer_count,
    AVG(frequency) as avg_frequency,
    AVG(recency_days) as avg_recency_days,
    AVG(monetary_value) as avg_monetary,
    AVG(avg_days_between_purchases) as avg_purchase_cycle_days
FROM rfm_labeled
GROUP BY rfm_segment
ORDER BY customer_count DESC;

In [0]:
-- 2.2 Basket Analysis - Category Affinity
WITH basket_items AS (
    SELECT 
        s.transaction_id,
        s.customer_key,
        p.category_level_2 as category,
        COUNT(*) as items_in_category,
        SUM(s.net_sales_amount) as category_revenue
    FROM juan_dev.retail.gold_sales_fact s
    JOIN juan_dev.retail.gold_product_dim p ON s.product_key = p.product_key
    WHERE s.is_return = false
    GROUP BY s.transaction_id, s.customer_key, p.category_level_2
),
category_pairs AS (
    SELECT 
        b1.category as category_1,
        b2.category as category_2,
        COUNT(DISTINCT b1.transaction_id) as co_occurrence_count,
        COUNT(DISTINCT b1.customer_key) as unique_customers,
        AVG(b1.category_revenue + b2.category_revenue) as avg_combined_revenue
    FROM basket_items b1
    JOIN basket_items b2 
        ON b1.transaction_id = b2.transaction_id 
        AND b1.category < b2.category  -- Avoid duplicates
    GROUP BY b1.category, b2.category
)
SELECT 
    category_1,
    category_2,
    co_occurrence_count,
    unique_customers,
    ROUND(avg_combined_revenue, 2) as avg_combined_revenue,
    ROUND(co_occurrence_count * 100.0 / (
        SELECT COUNT(DISTINCT transaction_id) 
        FROM juan_dev.retail.gold_sales_fact
    ), 2) as basket_percentage
FROM category_pairs
WHERE co_occurrence_count > 10
ORDER BY co_occurrence_count DESC
LIMIT 20;

# 3 🔄 Customer Engagement & Digital Behavior

In [0]:
-- 3.1 Multi-Channel Engagement Funnel
WITH customer_events AS (
    SELECT 
        e.customer_key,
        e.event_type,
        e.channel_key,
        ch.channel_category,
        COUNT(*) as event_count,
        SUM(CASE WHEN e.event_type = 'product_view' THEN 1 ELSE 0 END) as product_views,
        SUM(CASE WHEN e.event_type = 'add_to_cart' THEN 1 ELSE 0 END) as add_to_carts,
        SUM(CASE WHEN e.event_type = 'checkout_complete' THEN 1 ELSE 0 END) as purchases,
        COUNT(DISTINCT e.session_id) as sessions,
        AVG(e.time_on_page_seconds) as avg_time_on_page
    FROM juan_dev.retail.gold_customer_event_fact e
    JOIN juan_dev.retail.gold_channel_dim ch ON e.channel_key = ch.channel_key
    GROUP BY e.customer_key, e.event_type, e.channel_key, ch.channel_category
),
funnel_metrics AS (
    SELECT 
        channel_category,
        COUNT(DISTINCT customer_key) as unique_visitors,
        SUM(product_views) as total_product_views,
        SUM(add_to_carts) as total_add_to_carts,
        SUM(purchases) as total_purchases,
        -- Conversion rates
        SUM(add_to_carts) * 100.0 / NULLIF(SUM(product_views), 0) as view_to_cart_rate,
        SUM(purchases) * 100.0 / NULLIF(SUM(add_to_carts), 0) as cart_to_purchase_rate,
        SUM(purchases) * 100.0 / NULLIF(SUM(product_views), 0) as overall_conversion_rate,
        AVG(avg_time_on_page) as avg_engagement_time
    FROM customer_events
    GROUP BY channel_category
)
SELECT 
    channel_category,
    unique_visitors,
    total_product_views,
    total_add_to_carts,
    total_purchases,
    ROUND(view_to_cart_rate, 2) as view_to_cart_pct,
    ROUND(cart_to_purchase_rate, 2) as cart_to_purchase_pct,
    ROUND(overall_conversion_rate, 2) as overall_conversion_pct,
    ROUND(avg_engagement_time, 0) as avg_seconds_engaged
FROM funnel_metrics
ORDER BY unique_visitors DESC;

In [0]:
-- 3.2 Abandonment Recovery Analysis
WITH abandonment_cohorts AS (
    SELECT 
        DATE_TRUNC('week', d.calendar_date) as abandonment_week,
        ca.abandonment_stage,
        COUNT(*) as total_abandonments,
        SUM(ca.cart_value) as total_abandoned_value,
        AVG(ca.cart_value) as avg_cart_value,
        AVG(ca.items_count) as avg_items_abandoned,
        SUM(CASE WHEN ca.recovery_email_sent THEN 1 ELSE 0 END) as emails_sent,
        SUM(CASE WHEN ca.recovery_email_opened THEN 1 ELSE 0 END) as emails_opened,
        SUM(CASE WHEN ca.recovery_email_clicked THEN 1 ELSE 0 END) as emails_clicked,
        SUM(CASE WHEN ca.is_recovered THEN 1 ELSE 0 END) as carts_recovered,
        SUM(ca.recovery_revenue) as recovered_revenue
    FROM juan_dev.retail.gold_cart_abandonment_fact ca
    JOIN juan_dev.retail.gold_date_dim d ON ca.date_key = d.date_key
    GROUP BY DATE_TRUNC('week', d.calendar_date), ca.abandonment_stage
)
SELECT 
    abandonment_week,
    abandonment_stage,
    total_abandonments,
    ROUND(total_abandoned_value, 2) as total_abandoned_value,
    ROUND(avg_cart_value, 2) as avg_cart_value,
    ROUND(avg_items_abandoned, 1) as avg_items,
    -- Email performance
    ROUND(emails_opened * 100.0 / NULLIF(emails_sent, 0), 1) as email_open_rate,
    ROUND(emails_clicked * 100.0 / NULLIF(emails_opened, 0), 1) as email_click_rate,
    -- Recovery metrics
    ROUND(carts_recovered * 100.0 / NULLIF(total_abandonments, 0), 1) as recovery_rate,
    ROUND(recovered_revenue, 2) as recovered_revenue,
    ROUND(recovered_revenue * 100.0 / NULLIF(total_abandoned_value, 0), 1) as revenue_recovery_rate
FROM abandonment_cohorts
ORDER BY abandonment_week DESC, total_abandonments DESC;

# 4. 👥 Customer Product Affinity & Personalization

In [0]:
-- 4.1 Personalization Effectiveness Analysis
WITH affinity_performance AS (
    SELECT 
        cpa.customer_key,
        c.segment,
        c.preferred_category,
        COUNT(DISTINCT cpa.product_key) as products_with_affinity,
        AVG(cpa.affinity_score) as avg_affinity_score,
        MAX(cpa.affinity_score) as max_affinity_score,
        SUM(cpa.purchase_count) as total_purchases,
        SUM(cpa.view_count) as total_views,
        AVG(cpa.view_to_purchase_ratio) as avg_conversion,
        AVG(cpa.predicted_cltv_impact) as avg_cltv_impact
    FROM juan_dev.retail.gold_customer_product_affinity_agg cpa
    JOIN juan_dev.retail.gold_customer_dim c ON cpa.customer_key = c.customer_key
    WHERE c.is_current = true
    GROUP BY cpa.customer_key, c.segment, c.preferred_category
),
affinity_segments AS (
    SELECT 
        segment,
        preferred_category,
        COUNT(*) as customer_count,
        AVG(products_with_affinity) as avg_products_per_customer,
        AVG(avg_affinity_score) as overall_avg_affinity,
        AVG(total_purchases) as avg_purchases,
        AVG(total_views) as avg_views,
        AVG(avg_conversion) * 100 as avg_conversion_rate,
        AVG(avg_cltv_impact) as avg_predicted_cltv
    FROM affinity_performance
    GROUP BY segment, preferred_category
)
SELECT 
    segment,
    preferred_category,
    customer_count,
    ROUND(avg_products_per_customer, 1) as avg_products_interested,
    ROUND(overall_avg_affinity, 3) as avg_affinity_score,
    ROUND(avg_purchases, 1) as avg_purchases,
    ROUND(avg_views, 1) as avg_product_views,
    ROUND(avg_conversion_rate, 2) as conversion_rate_pct,
    ROUND(avg_predicted_cltv, 2) as predicted_cltv_impact
FROM affinity_segments
ORDER BY customer_count DESC;

In [0]:
-- 4.2 Cross-Category Purchase Behavior
WITH customer_categories AS (
    SELECT 
        s.customer_key,
        p.category_level_1,
        p.category_level_2,
        COUNT(DISTINCT s.transaction_id) as category_purchases,
        SUM(s.net_sales_amount) as category_revenue,
        AVG(s.net_sales_amount) as avg_order_value
    FROM juan_dev.retail.gold_sales_fact s
    JOIN juan_dev.retail.gold_product_dim p ON s.product_key = p.product_key
    WHERE s.is_return = false
    GROUP BY s.customer_key, p.category_level_1, p.category_level_2
),
category_diversity AS (
    SELECT 
        customer_key,
        COUNT(DISTINCT category_level_1) as l1_categories,
        COUNT(DISTINCT category_level_2) as l2_categories,
        SUM(category_purchases) as total_purchases,
        SUM(category_revenue) as total_revenue,
        MAX(category_revenue) / NULLIF(SUM(category_revenue), 0) as revenue_concentration
    FROM customer_categories
    GROUP BY customer_key
)
SELECT 
    CASE 
        WHEN l2_categories >= 5 THEN 'Highly Diverse'
        WHEN l2_categories >= 3 THEN 'Moderately Diverse'
        WHEN l2_categories = 2 THEN 'Limited Diversity'
        ELSE 'Single Category'
    END as customer_diversity,
    COUNT(*) as customer_count,
    AVG(l1_categories) as avg_l1_categories,
    AVG(l2_categories) as avg_l2_categories,
    AVG(total_purchases) as avg_purchases,
    AVG(total_revenue) as avg_revenue,
    AVG(revenue_concentration) * 100 as avg_revenue_concentration_pct
FROM category_diversity
GROUP BY 1
ORDER BY customer_count DESC;

# 5. 📈 Customer Lifecycle & Retention

In [0]:
-- 5.1 Cohort Retention Analysis
WITH customer_cohorts AS (
    SELECT 
        c.customer_key,
        DATE_TRUNC('month', c.acquisition_date) as cohort_month,
        c.acquisition_channel
    FROM juan_dev.retail.gold_customer_dim c
    WHERE c.is_current = true
),
customer_activity AS (
    SELECT 
        cc.customer_key,
        cc.cohort_month,
        cc.acquisition_channel,
        DATE_TRUNC('month', d.calendar_date) as activity_month,
        SUM(s.net_sales_amount) as monthly_revenue
    FROM customer_cohorts cc
    JOIN juan_dev.retail.gold_sales_fact s ON cc.customer_key = s.customer_key
    JOIN juan_dev.retail.gold_date_dim d ON s.date_key = d.date_key
    WHERE s.is_return = false
    GROUP BY cc.customer_key, cc.cohort_month, cc.acquisition_channel, DATE_TRUNC('month', d.calendar_date)
),
retention_calc AS (
    SELECT 
        cohort_month,
        acquisition_channel,
        activity_month,
        date_diff(MONTH, cohort_month, activity_month) as months_since_acquisition,
        COUNT(DISTINCT customer_key) as active_customers,
        SUM(monthly_revenue) as cohort_revenue
    FROM customer_activity
    GROUP BY cohort_month, acquisition_channel, activity_month
)
SELECT 
    cohort_month,
    acquisition_channel,
    months_since_acquisition,
    active_customers,
    FIRST_VALUE(active_customers) OVER (
        PARTITION BY cohort_month, acquisition_channel 
        ORDER BY months_since_acquisition
    ) as cohort_size,
    ROUND(active_customers * 100.0 / FIRST_VALUE(active_customers) OVER (
        PARTITION BY cohort_month, acquisition_channel 
        ORDER BY months_since_acquisition
    ), 1) as retention_rate,
    ROUND(cohort_revenue, 2) as monthly_revenue,
    ROUND(cohort_revenue / NULLIF(active_customers, 0), 2) as revenue_per_customer
FROM retention_calc
WHERE months_since_acquisition <= 12
ORDER BY cohort_month DESC, acquisition_channel, months_since_acquisition;

In [0]:
-- 5.2 Customer Reactivation Opportunities
WITH customer_last_purchase AS (
    SELECT 
        c.customer_key,
        c.customer_id,
        c.email,
        c.segment,
        c.lifetime_value,
        MAX(d.calendar_date) as last_purchase_date,
        DATEDIFF(CURRENT_DATE, MAX(d.calendar_date)) as days_since_purchase,
        COUNT(DISTINCT s.transaction_id) as total_orders,
        SUM(s.net_sales_amount) as total_spent,
        AVG(s.net_sales_amount) as avg_order_value
    FROM juan_dev.retail.gold_customer_dim c
    JOIN juan_dev.retail.gold_sales_fact s ON c.customer_key = s.customer_key
    JOIN juan_dev.retail.gold_date_dim d ON s.date_key = d.date_key
    WHERE c.is_current = true
        AND s.is_return = false
    GROUP BY c.customer_key, c.customer_id, c.email, c.segment, c.lifetime_value
),
customer_status AS (
    SELECT 
        *,
        CASE 
            WHEN days_since_purchase <= 30 THEN 'Active'
            WHEN days_since_purchase <= 90 THEN 'Cooling'
            WHEN days_since_purchase <= 180 THEN 'At Risk'
            WHEN days_since_purchase <= 365 THEN 'Dormant'
            ELSE 'Lost'
        END as customer_status,
        -- Calculate expected purchase frequency
        total_orders * 30.0 / NULLIF(
            DATEDIFF(last_purchase_date, 
                (SELECT MIN(calendar_date) FROM juan_dev.retail.gold_date_dim)
            ), 0) as expected_monthly_orders
    FROM customer_last_purchase
)
SELECT 
    customer_status,
    segment,
    COUNT(*) as customer_count,
    AVG(lifetime_value) as avg_ltv,
    AVG(total_orders) as avg_total_orders,
    AVG(total_spent) as avg_total_spent,
    AVG(avg_order_value) as avg_basket_size,
    AVG(days_since_purchase) as avg_days_inactive,
    SUM(lifetime_value) as total_ltv_at_risk,
    AVG(expected_monthly_orders) as avg_expected_monthly_orders
FROM customer_status
GROUP BY customer_status, segment
ORDER BY customer_status, customer_count DESC;

# 🎯 6. Size/Fit Behavior Analysis

In [0]:
-- 6.1 Size Fit Impact on Returns
WITH size_behavior AS (
    SELECT 
        c.customer_key,
        c.size_profile_tops,
        c.size_profile_bottoms,
        sf.ordered_size,
        sf.kept_size,
        sf.fit_description,
        sf.is_returned,
        p.category_level_2
    FROM juan_dev.retail.gold_size_fit_bridge sf
    JOIN juan_dev.retail.gold_customer_dim c ON sf.customer_key = c.customer_key
    JOIN juan_dev.retail.gold_product_dim p ON sf.product_key = p.product_key
    WHERE c.is_current = true
)
SELECT 
    category_level_2,
    fit_description,
    COUNT(*) as order_count,
    SUM(CASE WHEN is_returned THEN 1 ELSE 0 END) as returns,
    ROUND(SUM(CASE WHEN is_returned THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 1) as return_rate,
    COUNT(DISTINCT customer_key) as unique_customers,
    -- Size ordering patterns
    SUM(CASE WHEN ordered_size = kept_size THEN 1 ELSE 0 END) as correct_size_orders,
    ROUND(SUM(CASE WHEN ordered_size = kept_size THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 1) as size_accuracy_rate
FROM size_behavior
GROUP BY category_level_2, fit_description
HAVING COUNT(*) > 10
ORDER BY order_count DESC;